In [1]:
import pandas as pd

df = pd.read_csv("SKSNXTR_head5.csv", index_col=False)

In [2]:
df

,일자,대량매매구분코드,정규시간외구분코드,종목코드,종목인덱스,체결번호,체결가격,체결수량,체결유형코드,체결일자,...,최종매도매수구분코드,LP보유수량,데이터구분,메세지일련번호,매수프로그램호가신고구분코드,매도프로그램호가신고구분코드,보드ID,세션ID,실시간상한가,실시간하한가
0,20231201,NaN,NaN,KR572501BA28,30,1,1090,19900,NaN,20231201,...,1,0,1,18159183,NaN,NaN,G1,40,0,0
1,20231201,NaN,NaN,KR572501BA28,30,2,1090,7100,NaN,20231201,...,1,0,1,18159184,NaN,NaN,G1,40,0,0
2,20231201,NaN,NaN,KR572501BA28,30,3,1090,23000,NaN,20231201,...,1,0,1,18159185,NaN,NaN,G1,40,0,0
3,20231201,NaN,NaN,KR5744017777,64,1,339,1,NaN,20231201,...,2,0,1,1696497,NaN,NaN,G1,40,0,0
4,20231201,NaN,NaN,KR5744017777,64,2,320,500,NaN,20231201,...,1,0,1,8181079,NaN,NaN,G1,40,0,0


In [16]:
import polars as pl

# 스트리밍 스캔 → 앞 20 행만 가져오기
sample = (
    pl.scan_csv("SKSNXTR_filtered.csv")   # 헤더가 이미 있으므로 옵션 생략
)

In [17]:
sample

In [18]:
import FinanceDataReader as fdr

listing = (
    pl.from_pandas(
        fdr.StockListing("KRX")[["ISU_CD", "Name"]]
    )
    .rename({"ISU_CD": "종목코드", "Name": "종목명"})
    .with_columns(pl.col("종목코드").cast(pl.Utf8))
    .unique(subset="종목코드")
    .lazy()
) 

In [19]:
listing

In [30]:
result = (
    sample.join(listing, on="종목코드", how="left")
    .drop_nulls("종목명") 
        .collect(streaming=True)
)

/var/folders/bs/f41tt40n4051glsbp8qq036c0000gp/T/ipykernel_97144/2321647073.py:4: DeprecationWarning: The argument `streaming` is deprecated and is being replaced by the `engine` argument.
  .collect(streaming=True)
/var/folders/bs/f41tt40n4051glsbp8qq036c0000gp/T/ipykernel_97144/2321647073.py:4: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  .collect(streaming=True)


In [31]:
result

종목코드,종목인덱스,체결가격,체결일자,체결시각,시가,종목명
str,i64,i64,i64,i64,i64,str
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
…,…,…,…,…,…,…
"""KYG5307W1015""",6177,2525,20231228,180030021,0,"""엘브이엠씨홀딩스"""
"""KYG5307W1015""",6177,2525,20231228,180030021,0,"""엘브이엠씨홀딩스"""
"""KYG5307W1015""",6177,2525,20231228,180030021,0,"""엘브이엠씨홀딩스"""
